In [18]:
import matplotlib.pyplot as plt

import requests
import json

import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

from IPython.core.display import HTML, display

# Getting the data from API

In [39]:
r = requests.post("https://ies-midterm.soulution.rocks/login", json = {"cuni": "85952280"})
r.text
codes = r.json()
codes

{'data': {'personal_code': 'ff486d25',
  'dataset_ids': ['5df3513c79',
   '11f651d1ee',
   '0a42bff0a1',
   '403c8fd1da',
   'f45ec2e9fc',
   '775352ba11']},
 'message': 'Hello, Bajgarová Barbora! Good luck with the midterm. GET your data set from /data/file_id endpoint; use the codes you received in the data response as the file_id parameter! Print the personal code in your notebook.'}

In [43]:
print(codes['data']['personal_code'])


ff486d25


In [122]:
dataset = []
for i in codes['data']['dataset_ids']:
        r = requests.get(f"https://ies-midterm.soulution.rocks/data/{i}")
        a = r.json()
        while a['message'] == 'Error [503] - Sorry, I just slipped and fell. Thank God, I am made of silicon. You need to work this out.':
            r = requests.get(f"https://ies-midterm.soulution.rocks/data/{i}")
            a = r.json()
        dataset.append(a)
            
dataset

[{'data': {'company': 'SLA',
   'data': [{'Date': '2019-01-02',
     'Open': '77.48999786376953',
     'High': '78.98999786376953',
     'Low': '76.83999633789062',
     'Close': '77.48999786376953',
     'Adj.Close': '77.48999786376953',
     'Volume': '20000'},
    {'Date': '2019-01-03',
     'Open': '77.48999786376953',
     'High': '77.48999786376953',
     'Low': '77.48999786376953',
     'Close': '77.48999786376953',
     'Adj.Close': '77.48999786376953',
     'Volume': '0'},
    {'Date': '2019-01-04',
     'Open': '77.48999786376953',
     'High': '77.48999786376953',
     'Low': '77.48999786376953',
     'Close': '77.48999786376953',
     'Adj.Close': '77.48999786376953',
     'Volume': '0'},
    {'Date': '2019-01-07',
     'Open': '81.2699966430664',
     'High': '83.19999694824219',
     'Low': '80.87000274658203',
     'Close': '81.2699966430664',
     'Adj.Close': '81.2699966430664',
     'Volume': '35050'},
    {'Date': '2019-01-08',
     'Open': '81.2699966430664',
     '

# Dataset analysis

## Creating one big dataframe, adding a company column and setting index as Date and disregarding NaN values

In [239]:
dataframe = pd.DataFrame()
company = []
for j in range(0,len(dataset)):
    df = pd.DataFrame(dataset[j]['data']['data']).set_index('Date')
    df['Company'] = dataset[j]['data']['company']
    company.append(dataset[j]['data']['company']) #this will be useful later
    dataframe = pd.concat([dataframe, df], axis = 0)
    
dataframe = dataframe.dropna()
dataframe = dataframe.sort_index()
dataframe

,Open,High,Low,Close,Adj.Close,Volume,Company
Date,,,,,,,
2019-01-02,77.48999786376953,78.98999786376953,76.83999633789062,77.48999786376953,77.48999786376953,20000,SLA
2019-01-02,31.459999084472656,32.650001525878906,31.049999237060547,32.47999954223633,31.963159561157227,5229500,AAL
2019-01-02,35.630001068115234,36.810001373291016,34.709999084472656,36.65999984741211,36.65999984741211,0,MNG
2019-01-02,2.940000057220459,3.25,2.880000114440918,3.1700000762939453,3.0997939109802246,1987100,CPG
2019-01-02,38.38999938964844,38.560001373291016,38.33000183105469,38.4900016784668,37.300086975097656,10100,SMIN
...,...,...,...,...,...,...,...
2020-11-19,26.290000915527344,27.049999237060547,26.139999389648438,27.030000686645508,27.030000686645508,0,MNG
2020-11-20,26.940000534057617,27.170000076293945,26.649999618530273,26.84000015258789,26.84000015258789,0,MNG
2020-11-20,12.800000190734863,12.90999984741211,12.460000038146973,12.529999732971191,12.529999732971191,58491100,AAL


## Checking for dates - even from the first company SLA we can see that we do not have the desired dates range in our sample

In [197]:
print(dataframe[dataframe['Company']=='SLA'].head(1),dataframe[dataframe['Company']=='SLA'].tail(1) )

                         Open               High                Low  \
Date                                                                  
2019-01-02  77.48999786376953  78.98999786376953  76.83999633789062   

                        Close          Adj.Close Volume Company  
Date                                                             
2019-01-02  77.48999786376953  77.48999786376953  20000     SLA                             Open                High   Low               Close  \
Date                                                                           
2020-04-29  59.880001068115234  60.400001525878906  57.0  59.880001068115234   

                     Adj.Close Volume Company  
Date                                           
2020-04-29  59.880001068115234   7495     SLA  


## Getting max and min 

In [198]:
for i in company:
    dt = dataframe[dataframe['Company']==i]
    print(i,dt['Adj.Close'].max(), dt['Adj.Close'].min())

SLA 84.69000244140625 47.939998626708984
MNG 45.31999969482422 14.930000305175781
SGRO 0.20999999344348907 0.1550000011920929
CPG 4.7656121253967285 0.608388364315033
AAL 9.890000343322754 10.020000457763672
SMIN 39.71310806274414 20.77176284790039


## what day was the highest number of shares traded 


In [216]:
dt = dataframe[['Volume']]
a = dt.groupby('Date').sum()
a[a['Volume']==a.max()] #not really sure why this does not work, but I tried making a boolean only true where was the maximum and the printing the line

ValueError: Can only compare identically-labeled Series objects

## what is the total of shares traded 


In [220]:
sum(float(dataframe['Volume'])) #never mind, I tried

TypeError: cannot convert the series to <class 'float'>

## for the company with the highest volume of shares traded over the whole period, plot their share price over time 


In [240]:
a = dataframe[['Company', 'Volume']].groupby('Company').sum()
#a[a['Volume'] == a.max()]
#plt.plot()

Volume    inf
dtype: float64